# Genetic Algorithm Source Code and Demonstration

This notebook presents a problem generic implementation of the genetic algorithm. The Python classes are extended for the problem specific formulation of the N-Queens problem.

Table of Contents:
* [Chromosome Base Class](#Chromosome)
* [Genetic Algorithm Base Class](#Genetic-Algorithm)
* [N-Queens Formulation](#N-Queens-Formulation)
* [8-Queens Demonstration](#8-Queens-Demonstration)
* [16-Queens Demonstration](#16-Queens-Demonstration)

## Base Class Impelementations

### Chromosome

In [1]:
class Chromosome:
    """ 
    Implements an abstract chromosome class data type
    """
    
    def crossover(self, other):
        """
        Abstract method to implement cross-over between two chromosomes.
        @param other - other chromosome
        @return child1, child2 - output of the chromosome's children resultant from crossover
        """
        pass
    
    
    def mutate(self):
        """
        Abstract method to implement mutate operation on a chromosome.
        @return mutated chromosome
        """
        pass
    
    
    def getFitness(self):
        """
        Abstract method to implement the fitness evaluation for the chromosome.
        @return fitness value
        """
        pass
    
    def clone(self):
        """
        Abstract method to clone an instance of a chromosome
        """
        pass
    
    def __str__(self):
        """
        Abstract method to generate a string representation of the chromosome for printing.
        """
        pass

### Genetic Algorithm

In [2]:
import numpy as np

class GeneticAlgorithm:
    """
    Defines an abstract class for genetic algorithm.  All methods are implemetned
    except for buildInitialPopulation, which must be tailored to the Chromosome class
    used.
    """
    
    def __init__(self, popSize, generations, cross, mutate):
        """
        Initialization method for genetic algorithm class
        @param popSize - population size
        @param generations - number of generations the algorithm will run
        @param cross - probability of a crossover occuring during reproduction
        @param mutate - probability of a mutation occuring following reproduction
        """
        
        # Set GA parameter class variables
        self.populationSize = popSize
        self.numGenerations = generations
        self.probC = cross
        self.probM = cross
        
        # Initialize list class variables for population and roulette wheel
        self.population = [None for i in range(0,popSize)]
        self.roulette_min = [0 for i in range(0,popSize)]
        self.roulette_max = [0 for i in range(0,popSize)]
        
    def buildInitialPopulation(self):
        """
        Abstract method to generate a population of chromosomes.
        """
        pass
    
    def calculateRoulette(self):
        """
        Constructs a roulette wheel for parent selection.
        """
        
        # Determine the total fitness
        sum = 0
        for chromosome in self.population:
            sum = sum + chromosome.getFitness()
        
        # Generates roulette wheel where roulette_max[i] - roulette_min[i] == chromosome[i].getFitness()
        self.roulette_min[0] = 0
        for i in range(0, self.populationSize):
            if i != 0:
                self.roulette_min[i] = self.roulette_max[i-1]
            self.roulette_max[i] = self.roulette_min[i] + self.population[i].getFitness() / sum


    def pickChromosome(self):
        """
        Using roulette wheel, returns the index of a parent for reproduction.
        @return index of chromosome to reproduce.
        """
        spin = np.random.uniform()
        for i in range(0,self.populationSize):
            if spin > self.roulette_min[i] and spin <= self.roulette_max[i]:
                return i
        return self.populationSize-1
    

    def reproductionLoop(self):
        """ 
        Implements the GA algorithm's reproduction loop.  It is called once per generation.
        """
        newPop = []

        # Look through population populationSize/2 times
        #  each iteration generates two children
        for i in range(0, self.populationSize, 2):
            
            # Clone parents - Python copies by reference so we want to
            #  make sure we do not update the parents by mistake.
            x = self.population[self.pickChromosome()].clone()
            y = self.population[self.pickChromosome()].clone()
            
            # Crossover given crossover probabilty
            if (np.random.uniform() < self.probC):
                x, y = x.crossover(y)                
            
            # Mutate given mutate probability for each child
            if (np.random.uniform() < self.probM):
                x.mutate()
                
            if (np.random.uniform() < self.probM):
                y.mutate()
            
            # Add Children to new population
            newPop.append(x)
            newPop.append(y)
            
        # Update GA population with new population
        self.population = newPop    
    
    
    def printResults(self):
        """
        Prints the results of the current generation.  
        @return best chromosome
        """
        
        best = self.population[0]
        fit = []
        
        for chromosome in self.population:
            fit.append(chromosome.getFitness())
            if chromosome.getFitness() > best.getFitness():
                best = chromosome
                
        print("Best State: " + str(best.getFitness()))
        return best
    
    
    def runGA(self, target=0):
        """
        Implements the main GA population loop
        """
        
        # Initialize Variables toTrack best by generation and overall
        best = None
        bestOverall = None
        
        # Build initial poulation
        self.buildInitialPopulation()
        for i in range(0, self.numGenerations):

            # Generate roulette wheel for current population
            self.calculateRoulette()
        
            # Execute the GA reproduction loop for this generation
            self.reproductionLoop()
        
            # print generation's fitness and get best chromosome
            best = self.printResults()
            
            # Track the best
            if bestOverall is None:
                bestOverall = best
            elif best.getFitness() > bestOverall.getFitness():
                bestOverall = best
            
            # If target is reached, end algorithm
            if best.getFitness() >= target:
                print("Solution found at generation " + str(i))
                break
        
        # Prints the best overall solution
        print("Best overall Solution")
        print("Fitness: " + str(bestOverall.getFitness()))
        print(bestOverall)

## N-Queens Implementation and Demonstration

### N-Queens Formulation

This implementation does not use bits.  It specifically uses an array of size n specifying the column in which a queen is located for each row of the board.  

Crossover is implemented at the gene level (queen level) by slicing and concatenating between the two chromosomes at a random row.

Muation is implemented at the gene level by selecting a random row and changing its column to be a value from 0 to n-1, specifing a new column for the queen to reside.

In [3]:
import numpy as np

class QueenChromosome(Chromosome):
    
    def __init__(self, n, state=None):
        """
        Initialzies the queen chromosome.  If a child, it will clone the parent.  
        If a new chromosome, the values are initialized from 0 to n-1 for each gene.
        
        @param n - number of queens
        @param state - optional, state of parent (if child)
        """
        self.state = []
        self.numQueens = n
        
        # build new state
        if state == None:
            for i in range(0,n):
                self.state.append(np.random.randint(0,n))
        # Clone parent 
        else:
            self.state = state[:]
            
        # Call calculate fitness method and store in chromosome. (saves computational time)
        self.calculateFitness()
        
    def clone(self):
        """
        Creates a new queen by passing its number of queens and state to a new 
        nodes constructor
        @return cloned copy of node.
        """
        return QueenChromosome(self.numQueens, self.state)
    
    def crossover(self, other): 
        """
        Implements crossover for our queens chromosome formulation
        @return child1, child2 - two child nodes from self and other cross-over.
        """
        
        # Get Crossover point
        point = np.random.randint(0,self.numQueens)
        
        # split self
        x1 = self.state[0:point]
        x2 = self.state[point:]
        
        # split other
        y1 = other.state[0:point]
        y2 = other.state[point:]
        
        # crossover by appending sublists
        child1 = QueenChromosome(self.numQueens, x1 + y2)
        child2 = QueenChromosome(self.numQueens, y1 + x2)
        
        # return children
        return child1, child2
    
    def mutate(self):
        """
        Implements mutation on the chromosome.
        """
        # get point for mutation
        point = np.random.randint(0,self.numQueens)
                                  
        # Assign a new value at mutation point                          
        oldValue = self.state[point]
        while (oldValue == self.state[point]):
            self.state[point] = np.random.randint(0, self.numQueens)

            
    def calculateFitness(self):
        """
        Determines fitness of queens state.  For each queen i, it starts with a fitness of n-i
        and 1 point is deducted per conflicting queen j.  This value is summed for all queens.
        
        Max fitness is n x n-1
        
        @ return fitness.
        """
    
        # intialize
        self.fitness = 0
        rowFitness  = 0
        
        # Determine each row's fitness
        for row in range(0,self.numQueens):
            rowFitness = self.numQueens - 1 #initalize row fitness
                                  
            # check other rows and diagonals                   
            col = self.state[row]
            for i in range(0, self.numQueens):
                
                if i == row:
                    continue
                
                # Check for other rows with same column
                if (self.state[i] == col):
                    rowFitness = rowFitness - 1
                #diagonal 1
                if (self.state[i] == (col + (i-row))):
                    rowFitness = rowFitness - 1
                #diagonal 2
                if (self.state[i] == (col + (i-row))):
                    rowFitness = rowFitness - 1
        
            # add row fitness to self fitness.
            self.fitness = self.fitness + rowFitness

    

    def getFitness(self):
        """
        Returns the chromosome's stored fitness value.
        """
        return self.fitness

    
    def __str__(self):
        """
        Converts the state into a string representation of an N x N chess board
        where N is defined by the desired number of queens.
        
        Queen represented by a "Q".  Empty represented by a "."
        
        @return string representation of the node's chess board state
        """
        boardStr = ""
        
        # Traverse row by row
        for i in range(0,self.numQueens):
            # Initialize row as a string list with "." for blank
            row = ["." for x in range(0,self.numQueens)]
            
            # If row has queen, update list at queen index with Q
            if self.state[i] != -1:
                row[self.state[i]] = "Q"
                
            # Convert list to a string and concatenate
            boardStr += " ".join(map(str, row)) + "\n"
        return boardStr
    
    
    
class QueensGA(GeneticAlgorithm):
    """
    Implements the GA algorithm for the Queen chromosome.
    """
    
    def __init__(self, popSize, generations, cross, mutate, numQueens=8):
        """
        Initialization method for genetic algorithm class
        @param popSize - population size
        @param generations - number of generations the algorithm will run
        @param cross - probability of a crossover occuring during reproduction
        @param mutate - probability of a mutation occuring following reproduction
        @param numQueens - number of queens
        """
        self.numQueens = numQueens
        super().__init__(popSize, generations, cross, mutate)
    
    def buildInitialPopulation(self):
        """
        Builds the initial population of queen chromosomes.
        """
        self.population = []
        for i in range(self.populationSize):
            self.population.append(QueenChromosome(self.numQueens))

### 8-Queens Demonstration

Demonstrates the 8-Queens puzzle using a GA.  It may require multiple executions for the solution to converge on its target.

In [4]:
population = 50
generations = 1000
p_crossover = 0.7
p_mutation = 0.1
n_queens = 8

fit_target = 8*7

q = QueensGA(population, generations, p_crossover, p_mutation, n_queens)
q.runGA(target=fit_target)

Best State: 50
Best State: 52
Best State: 52
Best State: 54
Best State: 50
Best State: 50
Best State: 52
Best State: 50
Best State: 50
Best State: 52
Best State: 50
Best State: 50
Best State: 50
Best State: 50
Best State: 50
Best State: 50
Best State: 52
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 52
Best State: 52
Best State: 52
Best State: 52
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 52
Best State: 52
Best State: 52
Best State: 52
Best State: 50
Best State: 50
Best State: 52
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 54
Best State: 50
Best State: 52
Best State: 50
Best State: 50
Best State: 50
Best State: 50
Best State: 52
Best State: 52
Best State: 50
Best State: 50
Best State: 50
Best State: 54
Best State: 50
Best State: 52
Best State: 52
Best State: 52
Best State: 52
Best State: 52
Best State

### 16-Queens Demonstration

Demonstration of 16-Queens.  Convergence under this problem formulation is not likely.

In [5]:
population = 50
generations = 1000
p_crossover = 0.7
p_mutation = 0.1
n_queens = 16
fit_target = 16*15

q = QueensGA(population, generations, p_crossover, p_mutation, n_queens)
q.runGA(target=fit_target)

Best State: 228
Best State: 224
Best State: 224
Best State: 222
Best State: 224
Best State: 226
Best State: 224
Best State: 222
Best State: 222
Best State: 222
Best State: 224
Best State: 222
Best State: 224
Best State: 228
Best State: 228
Best State: 228
Best State: 230
Best State: 228
Best State: 226
Best State: 222
Best State: 226
Best State: 226
Best State: 224
Best State: 222
Best State: 224
Best State: 222
Best State: 230
Best State: 218
Best State: 222
Best State: 218
Best State: 226
Best State: 224
Best State: 226
Best State: 228
Best State: 228
Best State: 228
Best State: 224
Best State: 222
Best State: 230
Best State: 224
Best State: 224
Best State: 220
Best State: 224
Best State: 224
Best State: 226
Best State: 228
Best State: 228
Best State: 236
Best State: 226
Best State: 224
Best State: 222
Best State: 224
Best State: 228
Best State: 224
Best State: 224
Best State: 226
Best State: 226
Best State: 226
Best State: 224
Best State: 226
Best State: 222
Best State: 220
Best Sta

Best State: 226
Best State: 226
Best State: 228
Best State: 220
Best State: 226
Best State: 226
Best State: 222
Best State: 224
Best State: 226
Best State: 224
Best State: 220
Best State: 222
Best State: 224
Best State: 230
Best State: 230
Best State: 232
Best State: 230
Best State: 228
Best State: 228
Best State: 226
Best State: 228
Best State: 230
Best State: 230
Best State: 222
Best State: 224
Best State: 224
Best State: 226
Best State: 226
Best State: 222
Best State: 224
Best State: 222
Best State: 228
Best State: 226
Best State: 224
Best State: 222
Best State: 222
Best State: 222
Best State: 224
Best State: 226
Best State: 226
Best State: 222
Best State: 226
Best State: 224
Best State: 226
Best State: 226
Best State: 224
Best State: 224
Best State: 224
Best State: 224
Best State: 228
Best State: 228
Best State: 228
Best State: 232
Best State: 224
Best State: 222
Best State: 224
Best State: 228
Best State: 232
Best State: 232
Best State: 230
Best State: 226
Best State: 226
Best Sta